# Enhance feasibility

Without more specific restrictions, the search space of Limeade is quite large since it just considers basic rules of the molecular structure, making it hard to generate reasonable molecules in practice. This notebook uses the commonly used morgan fingerprint as the measurement of feasibility, and shows that how to enhance this type of feasibility in Limeade by just adding two lines of commands.

This notebooks list some examples to show how to use Limeade to achieve practical requirements.

The required Python libraries used this notebook are as follows:
- `Limeade`: the package this notebook demonstrates. Limeade can encode molecule space with given requirements into mathematical equations and generate feasible solutions quickly.
- `rdkit`: used to plot generated molecules.
- `numpy`: used to load the dataset.

In [1]:
from Limeade import MIPMol
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

We define the following function to check the feasibility. A molecule is claimed as infeasible if it contains a Morgan fingerprint with radius 1 that occurs less than 5 times in ChEMBL. This function will also record these "strange" substructures for later use.

In [2]:
fps = np.load("data/chembl_fps.npy", allow_pickle=True).item()

# checking if mol has strange substructures,
# if it does, put all strange substructures into a dictionary
def has_chembl_substruct(mol, strange_parts):
    fpgen = AllChem.GetMorganGenerator(radius=1)
    ao = AllChem.AdditionalOutput()
    ao.CollectBitInfoMap()
    fp = fpgen.GetSparseCountFingerprint(mol, additionalOutput=ao)
    info = ao.GetBitInfoMap()
    res = True
    for bit in fp.GetNonzeroElements().keys():
        if bit not in fps:
            res = False
            idx = info[bit][0][0]
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, 1, idx)
            submol = Chem.PathToSubmol(mol, env, atomMap={})
            smiles = Chem.MolToSmiles(submol)
            if smiles not in strange_parts:
                strange_parts[smiles] = 1
            else:
                strange_parts[smiles] += 1
    return res

For a list of molecules, we count the number of feasible molecules using the following function:

In [3]:
# count the number of feasible molecules
# a dictionary of strange substructures is also returned
def count_feasible(mols):
    # dictionary of strange substructures
    strange_parts = {}
    # number of feasible molecules
    cnt = 0
    for mol in mols:
        mol.UpdatePropertyCache()
        cnt += has_chembl_substruct(mol, strange_parts)
    strange_parts = sorted(strange_parts.items(), key=lambda item: -item[1])
    return cnt, strange_parts

## Round 1: without excluding substructures

We first count how many feasible molecules generated without excluding any substructures.

In [4]:
# set the number of atoms and types of atoms
N = 20
Mol = MIPMol(atoms=["C", "N", "O", "S"], N_atoms=N)

# set the bounds for the number of each type of atom (optional)
lb = [N // 2, None, None, None]
ub = [None, N // 4, N // 4, N // 4]
Mol.bounds_atoms(lb, ub)

# set the bounds for number of double/triple bonds, and rings (optional)
Mol.bounds_double_bonds(None, N // 2)
Mol.bounds_triple_bonds(None, N // 2)
Mol.bounds_rings(None, 0)

mols = Mol.solve(NumSolutions=1000)

cnt, strange_parts = count_feasible(mols)

print("Number of feasible molecules:", cnt)
print("Strange substructures appearing among generated molecules:")
print(strange_parts)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-11


  0%|          | 0/10 [00:00<?, ?it/s]

Discarded solution information
Reset all parameters


100%|██████████| 10/10 [00:01<00:00,  5.00it/s]


Number of feasible molecules: 5
Strange substructures appearing among generated molecules:
[('CC(C)(C)N', 557), ('CC(C)(C)C', 335), ('CC(C)(N)O', 248), ('CC(C)(C)S', 243), ('CC(C)(C)O', 228), ('CC(C)(N)S', 205), ('CC(C)(N)N', 145), ('CC(C)(O)S', 126), ('CC(C)(S)S', 105), ('C=C(C)C', 105), ('CC(N)(O)S', 70), ('CC(C)(O)O', 65), ('CN=S', 59), ('CN=O', 52), ('CN(C)S', 48), ('NSO', 45), ('CC(C)=S', 44), ('CN(N)S', 43), ('CC(N)N', 39), ('CC(C)C', 36), ('CSS', 35), ('C=C(C)N', 32), ('OS', 26), ('CN=N', 26), ('CC(O)(S)S', 25), ('COS', 24), ('CN(C)C', 24), ('CC(N)(S)S', 23), ('CC(N)(O)O', 23), ('CN(N)O', 20), ('CC(C)=N', 20), ('CC(N)(N)S', 19), ('CN(C)O', 19), ('CC(N)(N)O', 18), ('CC(O)(O)S', 17), ('CC(N)O', 17), ('CC(S)(S)S', 13), ('CC(N)=S', 13), ('C=NC', 13), ('CN(C)N', 12), ('CSO', 12), ('CNS', 12), ('CC(O)(O)O', 12), ('CC(C)N', 11), ('CC(=S)S', 10), ('C=C(C)S', 10), ('CC(C)S', 10), ('CC(O)S', 9), ('CC(=O)S', 8), ('CC(N)S', 7), ('NOS', 6), ('ON=S', 6), ('NN=S', 5), ('NON', 5), ('NCS', 5), (

## Round 2: exclude carbon without implicit hydrogen

From those strange substructures as shown in Round 1, we can notice that most molecules are infeasible since they contain a carbon linked with four carbon atoms or heteroatoms. Let us try to exclude this substructure and check if we can get more feasible molecules.

In [5]:
# exclude substructure C&H0
Mol.exclude_substructures(["[C&H0]"])

mols = Mol.solve(NumSolutions=1000)

cnt, strange_parts = count_feasible(mols)

print("Number of feasible molecules:", cnt)
print("Strange substructures appearing among generated molecules:")
print(strange_parts)

100%|██████████| 10/10 [00:01<00:00,  5.45it/s]


Number of feasible molecules: 61
Strange substructures appearing among generated molecules:
[('CN(C)N', 429), ('CC(N)N', 337), ('CC(N)O', 236), ('CN(N)S', 194), ('CN(N)O', 192), ('CC(N)S', 184), ('CN(C)S', 160), ('CN(N)N', 159), ('CSN', 88), ('OS', 61), ('CN(O)S', 56), ('NC(O)O', 54), ('CC(O)S', 53), ('NON', 52), ('CN(C)O', 49), ('COS', 46), ('NOO', 42), ('NC(N)O', 33), ('CN(O)O', 33), ('NOS', 32), ('CON', 25), ('NN(O)S', 24), ('CSO', 24), ('CC(C)S', 21), ('CC=S', 20), ('NN(S)S', 19), ('NC(N)S', 15), ('NN(N)O', 14), ('NC(O)S', 13), ('CC(O)O', 11), ('CC(S)S', 10), ('CN(S)S', 10), ('NCS', 9), ('CNS', 8), ('NN(O)O', 7), ('NN(N)S', 7), ('NNN', 6), ('NN(N)N', 6), ('OOS', 6), ('NSS', 6), ('CN=S', 6), ('NCO', 5), ('NSO', 5), ('OSO', 4), ('NC(S)S', 4), ('NC(N)N', 4), ('NNO', 4), ('CSC', 3), ('SCS', 3), ('OC(O)S', 2), ('SNS', 2), ('SOS', 2), ('NSN', 2), ('OOO', 1), ('N=CN', 1), ('CN(C)C', 1), ('O=CS', 1), ('ONS', 1), ('S=CS', 1), ('N=NN', 1), ('ON(O)O', 1), ('OC(O)O', 1), ('CC(C)N', 1), ('NNS',

## Round 3: exclude chains of heteroatoms

In Round 2, we indeed get more feasible molecules than Round 1, but most molecules are still infeasible due to heteroatom-heteroatom or a carbon linked with two heteroatoms. We exclude both substructures and then see the improvements.

In [6]:
# exclude heteroatom-heteroatom or heteroatom-carbon-heteroatom
Mol.exclude_substructures(["[N,O,S]~[N,O,S]", "[N,O,S]~C~[N,O,S]"])

mols = Mol.solve(NumSolutions=1000)

cnt, strange_parts = count_feasible(mols)

print("Number of feasible molecules:", cnt)
print("Strange substructures appearing among generated molecules:")
print(strange_parts)

100%|██████████| 10/10 [00:03<00:00,  2.79it/s]

Number of feasible molecules: 777
Strange substructures appearing among generated molecules:
[('CC(C)S', 68), ('CC=S', 2)]


Now we have 777 feasible molecules. One might wonder that the total number of molecules is less than 1000. The reason is that we will remove duplicated molecules after solving the model. As shown in the next cell, we can generate 847 unique molecules after generating 1000 solutions, and 777 among 847 molecules are feasible. 

In [7]:
print("Number of unique molecules:", len(mols))

Number of unique molecules: 847


Let us see the performance after adding the aforementioned constraints in larger scale.

In [8]:
mols = Mol.solve(NumSolutions=100000)

cnt, strange_parts = count_feasible(mols)

print("Number of feasible molecules:", cnt)
print("Strange substructures appearing among generated molecules:")
print(strange_parts)

100%|██████████| 1000/1000 [06:47<00:00,  2.46it/s]


Number of feasible molecules: 56336
Strange substructures appearing among generated molecules:
[('CC(C)S', 7349), ('CC=S', 1115), ('CSC', 147), ('CC(C)N', 105), ('CN(C)C', 11), ('C=NC', 3)]


In [9]:
print(len(mols))

64806
